In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd drive/MyDrive/to_Sedat_for_testing/Teknofest_Sınıflandırma_DCM/Oturum_1

[Errno 2] No such file or directory: 'drive/MyDrive/to_Sedat_for_testing/Teknofest_Sınıflandırma_DCM/Oturum_1'
/content/drive/MyDrive/to_Sedat_for_testing/Teknofest_Sınıflandırma_DCM/Oturum_1


In [ ]:
!pwd

/content/drive/My Drive/to_Sedat_for_testing/Teknofest_Sınıflandırma_DCM/Oturum_1


In [ ]:
%cd ..

/content/drive/My Drive/to_Sedat_for_testing/Teknofest_Sınıflandırma_DCM


In [ ]:
!pip install pillow

In [ ]:
import cv2
import os
import pydicom
import time
import numpy as np 

inputdir = './Oturum_1/'
outdir = './Oturum_1_Output_Ready_To_TEST/'

def dicom2png():
  test_list = [ f for f in  os.listdir(inputdir)]

  i=0
  for f in test_list:
      ds = pydicom.read_file(inputdir + f) 
      img = ds.pixel_array 
      cv2.imwrite(outdir + f.replace('.dcm','.png'),img)
      i+=1
      if i%50==0:
        print("Sleepping 1 sec")
        time.sleep(1)



In [ ]:
def focus_actual_image(image):
    
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    
    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        break
    
    return ROI,x,y,w,h


def png2ROI():
  i=0
  for image in os.listdir(outdir):
      file_name   = image.split(".")[0]
      ROI,x,y,w,h = focus_actual_image(outdir+"/{}".format(image))
      try:
          mask = np.asarray(Image.open(outdir+"/{}.png".format(file_name)))
          roi_mask= mask[y:y+h, x:x+w]
          #print(roi_mask.shape)
          roi_mask = Image.fromarray(roi_mask)
          #print(type(roi_mask))
          roi_mask.save("./{}/{}.png".format(outdir,file_name))
      except :
          print("bir hata oluştu")
      
      i+=1
      if i%50 ==0 :
          time.sleep(1)


In [ ]:
png2ROI()

In [ ]:
target_size = 512  # 512x512

def padding_zeros():

    for image in os.listdir(outdir):
        file_name   = image.split(".")[0]
        img = Image.open(outdir + file_name + ".png")
        width, height = img.size
       
        if width < 512  or height < 512:
          left_side = int((target_size - width) / 2)
          top_side = int((target_size - height) / 2)
          #print(img.mode)

          result = Image.new(img.mode, (target_size, target_size), (0))
          result.paste(img, (left_side, top_side))
          result.save("{}{}.png".format(outdir,file_name))
        else:
          #print("else")
          img = img.resize((512,512))
          img.save("{}{}.png".format(outdir,file_name))

In [ ]:
padding_zeros()